#Milestone Project 1 :- Food Vsison Big 🍔👁️

The goal of this notebook is to beet the [DeepFood](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://arxiv.org/ftp/arxiv/papers/1606/1606.05675.pdf) paper, by beating the top-1 accuracy 77.4%


In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

## Importing Food101 data from tensorflow datasets

In [2]:
(train_data, test_data), ds_info = tfds.load('food101',
                                             split=['train', 'validation'],
                                             shuffle_files=True,
                                             with_info=True,
                                             as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteCH19MZ/food101-train.tfrecord*...:   0%|          …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteCH19MZ/food101-validation.tfrecord*...:   0%|     …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [3]:
train_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [4]:
class_names = ds_info.features['label'].names
class_names

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheesecake',
 'cheese_plate',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

##Preprocessing the data

Our train_data and test_data are in a tf.Data style. Lets convert them to tensors and batches by creating our own preprocessing functions

In [5]:
train_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

We need to implement the follwoing :-   
- Resize each image
- Change datatype
- Convert to batches

In [6]:
def preprocess_image(img, label, image_size=224):
  img = tf.cast(img, dtype=tf.float32)
  img = tf.image.resize(img, [image_size, image_size])
  return img, label

In [7]:
#Convert train_data to batches and tensors
train_data = train_data.map(map_func=preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE).batch(32)

#Convert train_data to batches and tensors
test_data = test_data.map(map_func=preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE).batch(32)

In [8]:
train_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

Great! Our dataset is now in batches, and in proper shape and datatype

## Create feature extraction model

In [9]:
#Lets setup mixed precision
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
mixed_precision.set_global_policy('mixed_float16')

In [10]:
#Create our base_model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

#Create inputs and outputs
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs)
x = GlobalAveragePooling2D()(x)
x = Dense(101)(x)
outputs = tf.keras.layers.Activation('softmax', dtype=tf.float32)(x)

#Create model
model = tf.keras.Model(inputs, outputs)

16705208/16705208 [==============================] - 2s 0us/step


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
 activation (Activation)     (None, 101)               0         
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_____________________________________________

## Fiiting and evaluating

In [12]:
#Compile the model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [32]:
#Creating callbacks
checkpoint_path_feature_extract = '/content/Checkpoints/feature_extract_model.ckpt'
model_checkpoint_feature_extract = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_feature_extract,
                                                                      monitor='val_accuracy',
                                                                      save_weights_only=True,
                                                                      save_best_only=True,
                                                                      verbose=1)

tensorboard_callback_feature_extract = tf.keras.callbacks.TensorBoard(log_dir='Experiments/feature_extract_model')

In [33]:
#Fit the model (remeber to save to history)
history_feature_extract = model.fit(train_data,
                                    epochs=3,
                                    steps_per_epoch=len(train_data),
                                    validation_data=test_data,
                                    validation_steps=int(0.15*len(test_data)),
                                    callbacks=[model_checkpoint_feature_extract,
                                               tensorboard_callback_feature_extract])

Epoch 1/3
2368/2368 [==============================] - ETA: 0s - loss: 1.1517 - accuracy: 0.6938
Epoch 1: val_accuracy improved from -inf to 0.74338, saving model to /content/Checkpoints/feature_extract_model.ckpt
2368/2368 [==============================] - 249s 104ms/step - loss: 1.1517 - accuracy: 0.6938 - val_loss: 0.9411 - val_accuracy: 0.7434
Epoch 2/3
2368/2368 [==============================] - ETA: 0s - loss: 0.9183 - accuracy: 0.7485
Epoch 2: val_accuracy improved from 0.74338 to 0.75212, saving model to /content/Checkpoints/feature_extract_model.ckpt
2368/2368 [==============================] - 246s 103ms/step - loss: 0.9183 - accuracy: 0.7485 - val_loss: 0.8998 - val_accuracy: 0.7521
Epoch 3/3
2367/2368 [============================>.] - ETA: 0s - loss: 0.7690 - accuracy: 0.7866
Epoch 3: val_accuracy improved from 0.75212 to 0.76165, saving model to /content/Checkpoints/feature_extract_model.ckpt
2368/2368 [==============================] - 236s 98ms/step - loss: 0.7690 - a

In [34]:
#Evaluate model on all test data
model.evaluate(test_data)

790/790 [==============================] - 73s 90ms/step - loss: 0.8808 - accuracy: 0.7579


[0.8808414340019226, 0.7578613758087158]

## Fine tuning the model

In [43]:
#Unfreeze some layers
base_model.trainable = True
for layer in base_model.layers[:-5]:
  layer.trainable = False

In [44]:
#Recompile
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [45]:
#Create new callbacks
#Creating callbacks
checkpoint_path_fine_tuned = 'Checkpoints/fine_tuned_model.ckpt'
model_checkpoint_fine_tuned = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_fine_tuned,
                                                                 monitor='val_accuracy',
                                                                 save_weights_only=True,
                                                                 save_best_only=True,
                                                                 verbose=1)

tensorboard_callback_fine_tuned = tf.keras.callbacks.TensorBoard(log_dir='Experiments/fine_tuned_model')

In [46]:
#Refit
history_fine_tuned = model.fit(train_data,
                               epochs=6,
                               steps_per_epoch=len(train_data),
                               initial_epoch=history_feature_extract.epoch[-1],
                               validation_data=test_data,
                               validation_steps=int(0.15*len(test_data)),
                               callbacks=[model_checkpoint_fine_tuned,
                                          tensorboard_callback_fine_tuned])

Epoch 3/6
2368/2368 [==============================] - ETA: 0s - loss: 0.6704 - accuracy: 0.8077
Epoch 3: val_accuracy improved from -inf to 0.75371, saving model to Checkpoints/fine_tuned_model.ckpt
2368/2368 [==============================] - 258s 104ms/step - loss: 0.6704 - accuracy: 0.8077 - val_loss: 0.9271 - val_accuracy: 0.7537
Epoch 4/6
2367/2368 [============================>.] - ETA: 0s - loss: 0.5518 - accuracy: 0.8427
Epoch 4: val_accuracy improved from 0.75371 to 0.76192, saving model to Checkpoints/fine_tuned_model.ckpt
2368/2368 [==============================] - 233s 98ms/step - loss: 0.5518 - accuracy: 0.8427 - val_loss: 0.9192 - val_accuracy: 0.7619
Epoch 5/6
2367/2368 [============================>.] - ETA: 0s - loss: 0.4734 - accuracy: 0.8646
Epoch 5: val_accuracy did not improve from 0.76192
2368/2368 [==============================] - 236s 98ms/step - loss: 0.4734 - accuracy: 0.8646 - val_loss: 0.9607 - val_accuracy: 0.7561
Epoch 6/6
2368/2368 [===================

In [47]:
#Evaluate the model after fine tuning
model.evaluate(test_data)

790/790 [==============================] - 70s 86ms/step - loss: 0.9357 - accuracy: 0.7580


[0.9357214570045471, 0.7580198049545288]

## View results on tensorboard

In [50]:
#Upload experiments and view
!tensorboard dev upload --logdir '/content/Experiments' --one_shot

2023-02-22 01:53:12.602850: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-22 01:53:12.603083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-22 01:53:12.603119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/jenggxNeSEWX5e5VGWUlHg/

[2023-02-22T01:53:14] Started scanning logdir.
E0222 01:

## Conclusion

Unfortunately we didn't beat the DeepFood paper. Next time :-
- Use a better version of EfficientNet
- Train for longer
- Run more experiments